# 1. load data

In [ ]:
# from langchain_community.document_loaders import PyPDFDirectoryLoader
#
# # 1.load data pdf
# DATA_PATH = "data"
#
#
# def load_documents():
#     document_loader = PyPDFDirectoryLoader(DATA_PATH)
#     return document_loader.load()
#
#
# documents = load_documents()
# print(f"total {len(documents)} documents")

In [ ]:
from langchain_community.document_loaders import UnstructuredWordDocumentLoader


# 1.load data
def load_documents():
    document_loader = UnstructuredWordDocumentLoader("data/sample-rfp.docx")
    return document_loader.load()


#running the step
documents = load_documents()
print(f"total {len(documents)} documents")
print(documents[0])

# 2. split data


In [ ]:
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_core.documents import Document


# 2. split data
def split_documents(documents: list[Document]):
    text_splitter = RecursiveCharacterTextSplitter(
        chunk_size=800,
        chunk_overlap=80,
        length_function=len,
        is_separator_regex=False,
    )
    return text_splitter.split_documents(documents)


#running the step
chunks = split_documents(documents)
print(f"Split {len(documents)} documents into {len(chunks)} chunks.")
print('----------------------')
print(chunks[0])

# 3. Generating Unique IDs for Each Chunk

In [ ]:
# 3. Generating Unique IDs for Each Chunk
def calculate_chunk_ids(chunks):
    # This will create IDs like "data/sample.docx:6:2"
    # Page Source : Page Number : Chunk Index

    last_page_id = None
    current_chunk_index = 0

    for chunk in chunks:
        source = chunk.metadata.get("source")
        page = chunk.metadata.get("page")
        current_page_id = f"{source}:{page}"

        # If the page ID is the same as the last one, increment the index.
        if current_page_id == last_page_id:
            current_chunk_index += 1
        else:
            current_chunk_index = 0

        # Calculate the chunk ID.
        chunk_id = f"{current_page_id}:{current_chunk_index}"
        last_page_id = current_page_id

        # Add it to the page meta-data.
        chunk.metadata["id"] = chunk_id

    return chunks


#running the step
for chunk in calculate_chunk_ids(chunks):
    print(f"chunk : {chunk}\n======================================================================")


# 4. embed data


In [ ]:
from langchain_ollama import OllamaEmbeddings


# 4. embed data
def get_embedding_function():
    embeddings = OllamaEmbeddings(model="llama3")
    return embeddings


#running the step
print(get_embedding_function())


# 5. reset db(optional)

In [ ]:
import os
import shutil


# 5. reset db(optional)
def clear_database():
    if os.path.exists("chroma"):
        shutil.rmtree("chroma")


#running the step
clear_database()

# 6. store in db

In [ ]:
from langchain_chroma import Chroma


# 6. store in db
def add_to_chroma(chunks: list[Document]):
    # Load the existing database.
    db = Chroma(
        persist_directory="chroma", embedding_function=get_embedding_function()
    )

    # Calculate Page IDs.
    chunks_with_ids = calculate_chunk_ids(chunks)

    # Add or Update the documents.
    existing_items = db.get(include=[])  # IDs are always included by default
    existing_ids = set(existing_items["ids"])
    print(f"Number of existing documents in DB: {len(existing_ids)}")

    # Only add documents that don't exist in the DB.
    new_chunks = []
    for chunk in chunks_with_ids:
        if chunk.metadata["id"] not in existing_ids:
            new_chunks.append(chunk)

    if len(new_chunks):
        print(f"👉 Adding new documents: {len(new_chunks)}")
        new_chunk_ids = [chunk.metadata["id"] for chunk in new_chunks]
        db.add_documents(new_chunks, ids=new_chunk_ids)
    else:
        print("✅ No new documents to add")


#running the step
add_to_chroma(chunks)

----------

# 7. ask


In [ ]:
from langchain_community.llms.ollama import Ollama
from langchain_core.prompts import ChatPromptTemplate


# 1. ask
def query_rag(query_text: str):
    # 2. Prepare the DB.
    embedding_function = get_embedding_function()
    db = Chroma(persist_directory="chroma", embedding_function=embedding_function)

    # 3. prompt template
    PROMPT_TEMPLATE = """
    Answer the question based only on the following context:

    {context}

    ---

    Answer the question based on the above context: {question}
    """

    # 4. Search the DB.
    results = db.similarity_search_with_score(query_text, k=5)

    # 5. generate complete prompt
    context_text = "\n\n---\n\n".join([doc.page_content for doc, _score in results])
    prompt_template = ChatPromptTemplate.from_template(PROMPT_TEMPLATE)
    prompt = prompt_template.format(context=context_text, question=query_text)
    # print(prompt)

    # 6.invoke llm
    model = Ollama(model="llama3")
    response_text = model.invoke(prompt)

    # 7. get the original source
    sources = [doc.metadata.get("id", None) for doc, _score in results]
    formatted_response = f"Response: {response_text}\nSources: {sources}"
    print(formatted_response)
    return response_text


#running the step
query_rag("What is this doc ?")

# 8. unit test

In [ ]:
from langchain_community.llms.ollama import Ollama

# 1. prompt template
EVAL_PROMPT = """
Expected Response: {expected_response}
Actual Response: {actual_response}
---
(Answer with 'true' or 'false') Does the actual response match the expected response?
"""


def test_monopoly_rules():
    assert query_and_validate(
        question="How much total money does a player start with in Monopoly? (Answer with the number only)",
        expected_response="$1500",
    )


def test_ticket_to_ride_rules():
    assert query_and_validate(
        question="How many points does the longest continuous train get in Ticket to Ride? (Answer with the number only)",
        expected_response="10 points",
    )

# 2. ask
def query_and_validate(question: str, expected_response: str):
    response_text = query_rag(question)
    prompt = EVAL_PROMPT.format(
        expected_response=expected_response, actual_response=response_text
    )

    # 3. invoke llm
    model = Ollama(model="llama3")
    evaluation_results_str = model.invoke(prompt)

    # 4. clean
    evaluation_results_str_cleaned = evaluation_results_str.strip().lower()

    print(prompt)

    # 5. check
    if "true" in evaluation_results_str_cleaned:
        # Print response in Green if it is correct.
        print("\033[92m" + f"Response: {evaluation_results_str_cleaned}" + "\033[0m")
        return True
    elif "false" in evaluation_results_str_cleaned:
        # Print response in Red if it is incorrect.
        print("\033[91m" + f"Response: {evaluation_results_str_cleaned}" + "\033[0m")
        return False
    else:
        raise ValueError(
            f"Invalid evaluation result. Cannot determine if 'true' or 'false'."
        )


In [ ]:
test_monopoly_rules()
test_ticket_to_ride_rules()